<a href="https://colab.research.google.com/github/MatteoStromieri/High-Performance-Computing-Project/blob/master/High_Performance_Computing_Benchmark_Plots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import files

# Upload CSV file
uploaded = files.upload()

# Get the first uploaded file (assuming it's a CSV)
file_name = list(uploaded.keys())[0]

# Read CSV into a DataFrame
df = pd.read_csv(file_name)

# Display the DataFrame
print("DataFrame from CSV:")
print(df)

# Plotting example (customize based on your data)
plt.figure(figsize=(10, 6))
plt.plot(df['Column1'], df['Column2'], label='Data')
plt.xlabel('X-Axis Label')
plt.ylabel('Y-Axis Label')
plt.title('Plot of CSV Data')
plt.legend()
plt.show()


In [ ]:
# chain generated data
n = 255
times = [0] * n
times_matrix = [[0] * 256 for _ in range(256)]

ccx = 0.15
cdd = 0.31
numa = 0.34
socket = 0.36
node = 0.65
different_nodes = 1.82

for i in range(n):
  j = i+1
  if i//4 == j//4:
    times[i] = ccx
  elif i//16 == j//16:
    times[i] = cdd
  elif i//32 == j//32:
    times[i] = numa
  elif i//64 == j//64:
    times[i] = socket
  elif i//128 == j//128:
    times[i] = node
  else:
    times[i] = different_nodes

for i in range(256):
  for j in range(256):
    if i == j:
      times_matrix[i][j] = 0
    elif i//4 == j//4:
      times_matrix[i][j] = ccx
    elif i//16 == j//16:
      times_matrix[i][j] = cdd
    elif i//32 == j//32:
      times_matrix[i][j] = numa
    elif i//64 == j//64:
      times_matrix[i][j] = socket
    elif i//128 == j//128:
      times_matrix[i][j] = node
    else:
      times_matrix[i][j] = different_nodes


In [ ]:
def get_time_chain(i):
  time = 0
  for j in range(1,i):
    step_j = [times[k] for k in range(max(0,j-1), min(i-2,j-1)+1)]
    time += max(step_j)
  return time

In [ ]:
chain_times = [0]*8
n_processes = [2,4,8,16,32,64,128,256]
index = 0

for i in n_processes:
  chain_times[index] = get_time_chain(i)
  index += 1

chain_times

[0.15,
 0.44999999999999996,
 1.21,
 2.7299999999999995,
 5.800000000000001,
 11.96000000000001,
 24.569999999999958,
 50.959999999999866]

In [ ]:
import math

def succ_left(p):
  h = math.floor(math.log2(p+1))
  if h == 0:
    i = 0
  else:
    i = 2*(p-2**h+1) + p//(2**(h+1)-2)
  return 2**(h+1) - 1 + i

def succ_right(p):
  h = math.floor(math.log2(p+1))
  if h == 0:
    i = 0
  else:
    i = 2*(p-2**h+1) + p//(2**(h+1)-2)
  return 2**h + 2**(h+1) - 1 + i

def get_time_btree(i):
  time = 0
  for j in range(1,math.floor(math.log2(i))+1):
    step_j = []
    for k in range(max(0,j-1), min(math.floor(math.log2(i))-1,j-1)+1):
      left = succ_left(k)
      right = succ_right(k)
      step_j.append(times_matrix[k][left]+times_matrix[k][right])
      #step_j.append(times_matrix[k][left])
      #step_j.append(times_matrix[k][right])
    time += max(step_j)
  return time

In [ ]:
btree_times = [0]*8
n_processes = [3,7,15,31,63,127,255,511]
index = 0

for i in n_processes:
  btree_times[index] = get_time_btree(i)
  index += 1

btree_times

[0.3, 0.76, 1.38, 2.0, 2.62, 3.24, 3.89, 4.54]

In [ ]:
def get_time_barrier_tree(p):
  time = 0
  h = int(math.log2(p))
  for i in range(1,h+1):
    step_j = [times_matrix[j][j+2**(i-1)] for j in range(0,p,2**i)]
    time += max(step_j)
  return 2*time

In [ ]:
# barrier - tree
barrier_tree_times = [0]*8
n_processes = [2,4,8,16,32,64,128,256]
index = 0

for i in n_processes:
  barrier_tree_times[index] = get_time_barrier_tree(i)
  index += 1
barrier_tree_times

[0.3, 0.6, 1.22, 1.8399999999999999, 2.52, 3.24, 4.54, 8.18]